## svd4test test

In [1]:
import ipyrad as ip
import ipyrad.analysis as ipa

### Load in an Assembly object with the necessary data

In [2]:
data = ip.Assembly("qtest")

  New Assembly: qtest


In [3]:
## local paths
oldata = "/home/deren/Dropbox/Public/PyRAD_TUTORIALS/tutorial_GBS/fastq/*"
oldphy = "/home/deren/Dropbox/Public/PyRAD_TUTORIALS/tutorial_GBS/outfiles/c85m4p3.phy"

## link to my assembly
data.link_fastqs(path=oldata)
data.outfiles.phylip = oldphy

    Linking to demultiplexed fastq files in '/home/deren/Dropbox/Public/PyRAD_TUTORIALS/tutorial_GBS/fastq/*'
    12 new Samples created in `qtest`.
    12 fastq files linked to 12 new Samples.


### create ipyclient
The launch will be incorporated into the function call eventually.

In [4]:
import ipyparallel as ipp
ipyclient = ipp.Client()

## Run SVD4tet test

In [5]:
data = ipa.svd4tet.main(data, ipyclient)

loading array
populating array with 495 quartets
  done


In [6]:
data._svd.path

'qtest_svd.h5'

In [51]:
outfile = open("test.svd.leaves", 'w')

for i in range(0, 1000, 10):
    chunk = [list(i) for i in io5["quartets"][i:i+10]]
    chunk = ["{},{}|{},{}".format(*i) for i in chunk]
    outfile.write(" ".join(chunk)+" ")
    
outfile.close()

In [29]:
import h5py

io5 = h5py.File(data._svd.path, 'r')

print \
"\n".join([",".join([list(i) for i in io5["quartets"][:]])


SyntaxError: invalid syntax (<ipython-input-29-e8195d7e2d79>, line 5)

In [6]:
data.outfiles.phylip

'/home/deren/Dropbox/Public/PyRAD_TUTORIALS/tutorial_GBS/outfiles/c85m4p3.phy'

In [6]:
%tb

CompositeError: one or more exceptions from call to method: worker
[3:apply]: TypeError: worker() takes exactly 2 arguments (1 given)

In [7]:
data._svd.path

'qtest_svd.h5'

In [ ]:
phylip = open(data.outfiles.phylip, 'r')
line = phylip.readline().strip().split()
ntax = int(line[0])
nbp = int(line[1])


In [19]:
import numpy as np
seqarray = np.zeros((ntax, nbp), dtype="S1")

In [23]:
import h5py
with h5py.File(data._svd.path, 'w') as io5:
    io5.create_dataset("seqarray", (ntax, nbp), dtype="S1")       
    for line, seq in enumerate(phylip.readlines()):
        seqarray[line] = np.array(list(seq.split()[-1]))

In [17]:
import itertools
qiter = itertools.combinations(range(len(data.samples)), 4)

In [11]:
nquarts = sum(1 for _ in qiter)
nquarts

495

In [12]:
if nquarts < 1000:
    chunk = nquarts

In [30]:
import h5py
import numpy as np

i = 0
with h5py.File(data._svd.path, 'w') as io5:
    io5.create_dataset("samples", (nquarts, 4), 
                       dtype=np.int16, chunks=(chunk, 4))
    io5.create_dataset("quartets", (nquarts, 4), 
                        dtype=np.int16, chunks=(chunk, 4))        
    io5.create_dataset("weights", (nquarts, 1), 
                        dtype=np.float16, chunks=(chunk, 1))
    dat = np.array(list(itertools.islice(qiter, 1000)))
    
    io5["samples"][i:i+dat.shape[0]] = dat
    print io5["samples"]

<HDF5 dataset "samples": shape (495, 4), type "<i2">


In [7]:
import h5py

io5 = h5py.File(data._svd.path, 'r')
io5["samples"][0:1000]

array([[ 0,  1,  2,  3],
       [ 0,  1,  2,  4],
       [ 0,  1,  2,  5],
       ..., 
       [ 7,  8, 10, 11],
       [ 7,  9, 10, 11],
       [ 8,  9, 10, 11]], dtype=int16)